# Market Segmentation using Attributed Graph Community Detection

**Writen by Atefeh Morsali



Market segmentation is the task of dividing a market into groups of customers with homogenous needs, such that
marketing firms can target groups and allocate resources efficiently, as customers in the same segment are likely to respond similarly to a given marketing strategy.

Traditional market segmentation methods are based on clustering attribute data, such as demographics (e.g., age, gender, ethnicity) and psychographic (e.g., lifestyle, personality) profiles, using traditional clustering algorithms (e.g., k-means).

Nowadays, social networks have become important in marketing, as social relationships can also impact the formation of market segments. For example, a marketing firm can use this information to design marketing campaigns that target influential users in the network. As a result, a user buys a product because one of his/her friends bought the same product.

In  this  project,  we  aim  to  find  market  segments  given  social  network  data.  These  social 
relations  can  be  captured  in  a  graph  framework  where  nodes  represent  customers/users  and edges 
represent some social relationship. The properties belonging to each customer/user can be treated as node 
attributes. Hence, market segmentation becomes the problem of community detection over attributed graphs, 
where the communities are formed based on graph structure as well as attribute similarities. For this, I implemented an algorithm from the paper “Community detection based on structural and attribute similarities" to find  the  relevant market segments and evaluated the obtained segments via influence propagation. 


In the following, I first provided a quick overview on comunity detection in general (section A), then discussed the methodology used in this project (based on paper [1]) to detect communities in attributed graph (section B). After that, I implemented a code to find relevant market segments using facebook  network  of  a  US university dataset as mentioned in section C.



### A. Community Detection Overview

A community is a set of vertices in a graph that are densely connected within each other and sparsely connected 
with the rest of the graph. Communities in social network represent social groups with vertices representing users and edges represent relationships (e.g., “friendship”) between a pair of users.


Community detection is the problem of partitioning a given graph into communities. Solving this problem involves:
1. Defining an  <b> objective function </b> to partition the graph into communities.
    - <font color='blue'>Modularity</font> (the most widely used obj fnc)
    
    It is the difference between the number of edges within the communities and the expected number of these edges in a random graph with the same degree distribution.
        
        $Q = \frac{1}{2m}\sum \limits _{v,w∈V}[A_{vw}-\frac{k_{v} k_{w}}{2m}]δ(v, w)$
        
        where $A$ is the adjacency matrix of the graph, $m$ is the number of edges, $k_{v}$ is the degree of vertex $v$ and $δ(i,j)$ is 1 if $i$ and $j$ belong to the same community and 0 otherwise.



2. Defining <b>“goodness” metrics </b> to evaluate the quality of the communities.
    - <font color='blue'> Density </font>; the ratio of edges to the number of possible edges
    - <font color='blue'> Conductance </font>; the fraction of edges that point outside the community
    - <font color='blue'> Clustering coefficient </font>; the ratio of closed triplets to all triplets
    
    
Optimizing modularity is an NP-complete problem, but greedy algorithms for this problem have been proposed (e.g., the Louvain method)

### B. Community Detection in Attributed Graphs

Traditional community detection methods take into account only the structural information of the graph. However, many real-world networks, such as social networks, are attributed. Considering both the structural and attribute information of the graph may allow us to detect more meaningful communities.

Based on paper [1], we redefine the community detection problem for attributed graphs. Solving this problem involves:
 - Defining an <b> objective function </b> to partition the attributed graph into communities.
     - <font color='blue'> Composite modularity.
 - Defining <b> “goodness” metrics </b> to evaluate the quality of the attributed communities.
     -  <font color='blue'> Similarity </font>.

<b> Goodness metrics. </b>

It needs to consider the degree of closeness of the vertices in terms of their attributes. Vertices in the same community are expected to have similar attributes. To measure attribute similarity:
 - For binary attributes, simple matching coefficient (i.e., ratio of matching attributes to all attributes) can be used .
 - For continuous attributes, similarity metric based on the Euclidean distance can be used.

<b> Objective Function. </b>

Traditional modularity does not take into account the attribute similarity between vertices. Thus, paper [1] introduced the composite modularity, a weighted combination of modularity and similarity, as objective function for community detection in attributed graphs as follow:

$Q = \sum \limits _{v}\sum \limits _{v,w∈V}α.(\frac{1}{2m}.[A_{vw}-\frac{k_{v} k_{w}}{2m}])  + (1 − α) . sim(v, w))$

where $α$ is the weighting factor, $0 ≤ α ≤ 1$.

Optimizing composite modularity is an NP-complete problem, but greedy algorithms for this problem have been proposed (e.g., the Structure-Attribute Clustering SAC1 algorithm based on the Louvain method).

#### SAC1 Algorithm

<font color='blue'>Input</font>: attributed graph G = (V, E, X)

<font color='blue'>Output</font>: set of communities C

(1) Initialize a community for each vertex v ∈ V.

(2) For each vertex v ∈ V, assign v to the community that yields the highest positive gain in composite modularity.
    - Repeat (2) until no further improvement can be achieved.
    - Obtain a set of communities

(3) Construct a new graph by aggregating the vertices in each community into a single meta-vertex.

(4) Repeat (2) and (3) until no further improvement can be achieved.


It is similar to the Louvain method except we use composite modularity in step (2)

### C. Datasets

The  dataset  contains  a  facebook  network  of  a  US university  (given  as  an  edgelist)  with each  node corresponding  to  a  user  profile  having  the following  attributes:  student/faculty  status,  gender,  major,
second  major,  dorm,  and  year information. The data used in this project is downsampled to 324 users and stored in two small datasets: fb_caltech_small_edgelist.txt and fb_caltech_small_attrlist.csv.
Unfortunately, the Facebook Data Team requested that Porter no longer distribute the dataset. It does not include 
the names of individual or even of any of the node attributes (they have been given integer ids), but Facebook seems
to be concerned.

### D. Implementation

In [77]:
import numpy as np
import math
import sys
from scipy import spatial

In [109]:

class SAC1(object):
    def __init__(self, data_path, alpha):
        self.data_path = data_path
        self.alpha = alpha
        
        
        
    def load_data(self):  #loading the queries into a list

        attr_dic={} # attribute dictionary for each node
        deg_n=[]  # degree of each node
        num_edges=0
        edges={} # a list of ((int,int),weight) pairs
        edges_of_node={}
        nodes=[]

        with open(self.data_path+'/fb_caltech_small_edgelist.txt','r') as f:
             for line in f:
                 node,adj=line.split()
                 if int(node) not in nodes:
                    nodes.append(int(node))
                 if int(adj) not in nodes:
                    nodes.append(int(adj))
                 try:
                    edges[(int(node),int(adj))]+=edges[(int(node),int(adj))]
                 except:
                    edges[(int(node),int(adj))]=1

             edges=[(k,v) for k,v in edges.items()]

             deg_n=[0 for n in nodes] 


             for e in edges:
                 num_edges +=e[1] 
                 deg_n[e[0][0]]+=e[1]
                 deg_n[e[0][1]]+=e[1]     
                 if e[0][0] not in edges_of_node:
                    edges_of_node[e[0][0]]=[e]
                 else:
                    edges_of_node[e[0][0]].append(e) 
                 if e[0][1] not in edges_of_node:
                    edges_of_node[e[0][1]]=[e]
                 elif e[0][0]!=e[0][1]:
                    edges_of_node[e[0][1]].append(e)

        with open(self.data_path+'/fb_caltech_small_attrlist.csv','r') as f:
             next(f)
             count=0
             for line in f:
                 splitted_line=[int(x) for x in line.rstrip('\n').split(',')]

                 attr_dic[count]=splitted_line
                 count +=1
        nodes.sort()


        return nodes,edges,edges_of_node,attr_dic,deg_n,num_edges

    
    
    def initializing(self):

        # reading the data from the given input files 
        nodes,edges, edges_of_node, attr_dic,deg_n,num_edges=self.load_data()
        communities=[node for node in nodes]

        w_n=[0 for node in nodes] #initially ther is no self loops    
        s_att_in_com=[0 for node in nodes] # initially attribute simil is 0
        return nodes,edges,edges_of_node,attr_dic,deg_n,num_edges,communities,w_n,s_att_in_com
    
    
    def make_initial_partition(self, deg_n,nodes,edges):   # initial partition and communities
   
        partition=[[node] for node in nodes]

        tot_deg_comm=[deg_n[node] for node in nodes]

        s_in_com=[0 for n in nodes]
        
        for e in edges:
            if e[0][0]==e[0][1]:
               s_in_com[e[0][0]]+=e[1]
               s_in_com[e[0][1]]+=e[1]

        return partition,tot_deg_comm,s_in_com



    
    
    '''
    the first phase of the algorithm
    '''    
    def cal_sim_attr(self, n1, n2, attr_dic):  #attribute similarity using cosine similarity
        d1=attr_dic[n1]
        d2=attr_dic[n2]
        result=1-spatial.distance.cosine(d1,d2)
        return result
    
    
   
    def calculate_modularity_gain(self, node,com, sum_G_i_nodes,num_edges,tot_deg_comm,deg_n,sim_attr):  
        #computes the modularity of moving node in comunity.


        delta_newman=(1/(2.0*num_edges))*(2*sum_G_i_nodes-(deg_n[node]/(2*num_edges))*tot_deg_comm[com])


       # delta_newman=(2*sum_G_i_nodes-(deg_n[node]/(num_edges))*tot_deg_comm[com])


        gain=self.alpha*delta_newman+(1-self.alpha)*sim_attr    

        return gain
    

    def first_phase(self, nodes, edges, edges_of_node, attr_dic, deg_n, num_edges, communities,w_n,s_att_in_com):
        # initial partition and communities
        best_partition,tot_deg_comm,s_in_com = self.make_initial_partition(deg_n,nodes,edges)

        i=0   
        while True:
            print ("i= %d"%i)

            improvement=0
            for node in nodes:
                node_community=communities[node]
                #the best default comminity is node itself
                best_community=node_community
                best_gain=0

                #print node
                node_in_part=best_partition[node_community]
               # best_partition[node_community].remove(node) 

                #this is to calculate the sum of the weights within a community
                best_shared_edges=0
                best_shared_attr=0
                for e in edges_of_node[node]:
                    if e[0][0]==e[0][1]:
                       continue
                    if e[0][0]==node and communities[e[0][1]]==node_community or e[0][1]==node and communities[e[0][0]]==node_community:
                       best_shared_edges+=e[1]


                s_in_com[node_community] -=2*(best_shared_edges+w_n[node])      
                tot_deg_comm[node_community]-=deg_n[node] 

                sim_attr=0          
                #if best_partition[node_community]:
                for n in node_in_part:
                      # if n != node:
                      sim_attr +=self.cal_sim_attr(n,node,attr_dic)
                best_shared_attr=sim_attr
                s_att_in_com[node_community]-=best_shared_attr 

                best_shared_attr=0
                #communities[node]=-1
                ncommunities={}

                for node1 in nodes: 
                    community=communities[node1]
                    if community in ncommunities:
                       continue
                    ncommunities[community]=1
                    sum_G_i_nodes=0 #number of linkes between node and and members of the community, equation 4 of the paper
                    sim_attr=0  #attribute similarity
                    for e in edges_of_node[node]:
                        if e[0][0]==e[0][1]:
                           continue
                        if e[0][0]==node and communities[e[0][1]]==community or e[0][1]==node and communities[e[0][0]]==community:
                           sum_G_i_nodes +=e[1]

                    for n in best_partition[community]:
                       # if n!=node:
                           sim_attr +=self.cal_sim_attr(n,node,attr_dic)
                   #bounding Qatt by dividing by the squared size of community  
                    sim_attr_adj=sim_attr/math.pow(len(best_partition[community]),2)
                    #more adjusting the similarity attribute by the dividing similarity attribute to number of communities
                    sim_attr_adj=sim_attr_adj/(len([c for c in best_partition if c]))      
                    # compute modularity gain by moving node to the community of its neighbor
                    gain=self.calculate_modularity_gain(node,community,sum_G_i_nodes,num_edges,tot_deg_comm,deg_n,sim_attr_adj)
                    if gain>best_gain:
                       best_community=community
                       best_gain=gain
                       best_shared_edges=sum_G_i_nodes
                       best_shared_attr=sim_attr
                       best_node=node1

                # removing the node from its community
                best_partition[node_community].remove(node)

                #inserting node to best community
                best_partition[best_community].append(node)

                communities[node]=best_community
                tot_deg_comm[best_community]+=deg_n[node] 
                s_in_com[best_community]+=2*(best_shared_edges+w_n[node])                
                s_att_in_com[best_community]+=best_shared_attr
                #by only having one improvement, improvement value becomes one and the process repeats
                if node_community !=best_community:
                   improvement =1
            i+=1           
            if not improvement or i==15: # either 15 iteration or no improvement 

                break

        return best_partition,s_in_com,tot_deg_comm,s_att_in_com

    
                                                                                                               
                                                                                                               
                                                                                                              
                                                                                                               
    '''
    the second phase of the algorithm
    '''                                                                                                          
           
    def find_cent_com(self, p,attr_dic):  #function to find the centroid of a community: node which is most similiar to every other node
        sim_node={}
        for i in range(len(p)):
            sim_node[p[i]]=0

        for i in range(len(p)):
            for j in range(len(p)):
                sim_node[p[i]]+=self.cal_sim_attr(p[i],p[j],attr_dic)
        return max(sim_node,key=sim_node.get)                    
                                                                                                               
                                                                                                              
    def second_phase(self, communities,partition,edges,attr_dic,s_att_in_com):

        old_s_att_in_com=s_att_in_com
        s_att_in_com=[0 for com in communities]
        new_attr_dic={} 
        # attribute within a community/partition is equal to
        # attribute of the centroid the community
        k=0 
        for p in partition:
            centroid=self.find_cent_com(p,attr_dic)
            new_attr_dic[k]=attr_dic[centroid]
            k+=1
        attr_dic=new_attr_dic

        old_edges=edges
        old_communities=communities

        nodes=[i for i in range(len(partition))]
        s_att_in_com=[0 for node in nodes]


        # relablling the communities
        communities=[]
        d={}
        i=0
        for community in old_communities:
            if community in d:
               communities.append(d[community])
            else: 
                d[community]=i
                communities.append(i)
                i+=1

        edges={}

        for e in old_edges:
            ei=communities[e[0][0]]
            ej=communities[e[0][1]]
            try:
               edges[(ei,ej)]+=e[1]
            except KeyError:
               edges[(ei,ej)]=e[1]
        edges=[(k,v) for k,v in edges.items()]

        num=0
        w_n=[0 for n in nodes]
        deg_n=[0 for n in nodes]
        edges_of_node={}
        for e in edges:
            num +=e[1]#this is just to check if number of edges stay the same, which must stay the same
            deg_n[e[0][0]]+=e[1]
            deg_n[e[0][1]]+=e[1]
            if e[0][0]==e[0][1]:
               w_n[e[0][0]]+=e[1]
            if e[0][0] not in edges_of_node:
               edges_of_node[e[0][0]]=[e]
            else:
               edges_of_node[e[0][0]].append(e)
            if e[0][1] not in edges_of_node:
               edges_of_node[e[0][1]]=[e]
            elif e[0][0]!=e[0][1]:
               edges_of_node[e[0][1]].append(e)

        print ("edges: %d"%num)
        print ("this is a check that  no of edges must stay constant")

        communities=[n for n in nodes]

        return nodes, edges, edges_of_node, deg_n, communities,w_n,attr_dic,s_att_in_com

             
                                                                                                               
   
    def calculate_modularity(self, partition,s_in_com,num_edges,tot_deg_comm,s_att_in_com):
        q=0
        m2=2.0*num_edges
        for i in range(len(partition)):
            q+=1/m2*s_in_com[i]-(tot_deg_comm[i]/m2)**2+s_att_in_com[i]
        return q

    
    def louvain_method(self, nodes, edges, edges_of_node, attr_dic, deg_n, num_edges, communities,w_n,s_att_in_com):

        best_partition=[]
        actual_partition=[]
        best_Q=-1
        j=0
        while True:
            print ("j= %d" %j)


            partition,s_in_com,tot_deg_comm,s_att_in_com=self.first_phase(nodes, edges, edges_of_node, attr_dic, deg_n, num_edges, communities,w_n,s_att_in_com)

            Q=self.calculate_modularity(partition,s_in_com,num_edges,tot_deg_comm,s_att_in_com)
            partition=[c for c in partition if c]

            #grouping previous nodes with new partition 

            if actual_partition:
               actual=[]
               for p in partition:
                   part=[]
                   for n in p:
                       part.extend(actual_partition[n])
                   actual.append(part)
               actual_partition=actual

            else:
               actual_partition=partition        

            if Q==best_Q:
               break

            nodes,edges,edges_of_node,deg_n,communities,w_n,attr_dic,s_att_in_com=self.second_phase(communities,partition,edges,attr_dic,s_att_in_com) 
            best_partition=partition
            best_Q=Q
            j+=1
    
        return (actual_partition,best_Q) 

    def main(self):
        nodes, edges, edges_of_node, attr_dic, deg_n, num_edges, communities,w_n,s_att_in_com=self.initializing()
    
        best_partition,best_Q=self.louvain_method(nodes, edges, edges_of_node, attr_dic, deg_n, num_edges, communities,w_n,s_att_in_com)   
       
        print("best_partition", best_partition)
        print ("length of best partition %d" %len(best_partition))
        fw=open('communities.text','w')
        for part in best_partition:
            #fw.write("%s\n" %part)
            fw.write(str(part)[1:-1]+'\n')
        fw.close() 

In [110]:
Sac_1 = SAC1('data', 1)
Sac_1.main()

j= 0
i= 0
i= 1
i= 2
i= 3
edges: 6005
this is a check that  no of edges must stay constant
j= 1
i= 0
i= 1
edges: 6005
this is a check that  no of edges must stay constant
j= 2
i= 0
edges: 6005
this is a check that  no of edges must stay constant
j= 3
i= 0
best_partition [[10, 109], [9, 22, 41, 49, 51, 91, 101, 138, 163, 173, 174, 179, 185, 243, 248, 266], [0, 2, 7, 13, 15, 16, 17, 19, 21, 24, 28, 33, 34, 36, 37, 44, 45, 47, 50, 52, 53, 54, 57, 60, 61, 64, 68, 69, 72, 73, 76, 87, 95, 96, 98, 100, 104, 106, 110, 112, 113, 119, 120, 121, 122, 125, 126, 127, 133, 135, 136, 141, 142, 144, 147, 151, 153, 154, 155, 157, 158, 159, 167, 168, 169, 177, 180, 186, 189, 190, 193, 194, 196, 198, 199, 200, 202, 203, 204, 205, 206, 209, 210, 213, 214, 218, 221, 222, 223, 229, 230, 232, 235, 237, 242, 244, 250, 252, 258, 259, 261, 268, 270, 274, 279, 280, 282, 284, 289, 292, 296, 297, 298, 308, 310, 313, 315, 316, 323], [1, 4, 5, 8, 11, 12, 18, 27, 29, 35, 38, 39, 42, 43, 48, 56, 59, 62, 65, 66, 71, 74,

### E. Cluster Evaluation

To Evaluate the quality of the communities, I will use the concept of influence propagation. It has  been  discovered  that  by  targeting  the  influential  users/groups, desirable  marketing  goals  can  be  achieved.  Therefore,  one  way  to  evaluate  the  quality  of  the market segments (communities) is to influence an entity in each segment and measure how fast the influence propagates over the entire network. The faster that influence propagates through the entire network, the more likely an advertising campaign, for example, will be successful.
 
The code will measure and compare the time steps taken by the influence propagation algorithm to achieve maximum propagation and compare this with kmeans clustering. 




### Conclusion

### Reference
[1] T. A. Dang, and E. Viennet. Community detection based on structural and attribute similarities. ICDS 2012.

[2] CSC 591 lecture notes. Algorithm for Data Guided Business Intelligence, NCSU.